# Local Churn Scoring Workflow

**Complete end-to-end workflow for local scoring and exploration.**

This notebook provides all local processing utilities:
- **Optional:** Split large CSV files into chunks
- **Load** CSV files (single file or directory)
- **Score** customers with churn predictions (uses `function_app/scorer.py`)
- **Shape** data to match SQL view structure
- **Explore** results interactively
- **Export** outputs for Power BI exploration

**Inputs:** CSV files from `data/` folder (or single file)

**Outputs:**
- `outputs/churn_scores_combined.csv` - All scored records
- `outputs/churn_scores_sql_view.csv` - Shaped to match SQL view
- `outputs/README.md` - Project documentation
- `outputs/model_conda.yml` - Model environment configuration

## 0) Environment Setup

In [ ]:
import sys
import platform
from pathlib import Path

print("Python:", sys.version)
print("Platform:", platform.platform())

# Set project root (notebook is in scripts/)
PROJECT_ROOT = Path().resolve().parent
print(f"\nProject root: {PROJECT_ROOT}")

### Optional: Split Large CSV Files

If you have a very large CSV file that needs to be split into smaller chunks:

In [ ]:
# Optional: Split large CSV file into smaller chunks
# Uncomment and adjust if you need to split a large CSV file first

# LARGE_CSV_FILE = PROJECT_ROOT / "path" / "to" / "large_file.csv"

def split_csv(input_file, rows_per_file=50000, output_dir="data"):
    """Split a CSV file into smaller chunks."""
    import os
    from typing import cast
    import pandas as pd  # Import pandas inside function since it's defined before imports cell
    from pandas import DataFrame
    
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Reading {input_file}...")
    # pd.read_csv with file path and low_memory=False always returns DataFrame
    # Use cast to inform type checker of the expected type
    df = cast(DataFrame, pd.read_csv(input_file, low_memory=False))
    total_rows = len(df)
    print(f"Total rows: {total_rows:,}")
    
    num_chunks = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file else 0)
    print(f"Splitting into {num_chunks} files of up to {rows_per_file:,} rows each...")
    
    base_name = Path(input_file).stem
    
    for i in range(num_chunks):
        start_idx = i * rows_per_file
        end_idx = min((i + 1) * rows_per_file, total_rows)
        
        chunk = df.iloc[start_idx:end_idx]
        output_file = Path(output_dir) / f"{base_name}_part{i+1:03d}.csv"
        
        chunk.to_csv(output_file, index=False)
        print(f"  ✓ Wrote {output_file.name} ({len(chunk):,} rows)")
    
    print(f"\n✓ Done! {num_chunks} files created in {output_dir}/")
    return num_chunks

# Uncomment to run (adjust path as needed):
# split_csv(LARGE_CSV_FILE, SPLIT_ROWS_PER_FILE, SPLIT_OUTPUT_DIR)

## 1) Imports

In [ ]:
import pandas as pd
import numpy as np
import shutil
import time
from datetime import datetime

# Add function_app to path for imports
sys.path.insert(0, str(PROJECT_ROOT / "function_app"))
from scorer import score_customers, load_model  # type: ignore[import-untyped]

print("✓ Imports successful")

## 2) Configuration

In [ ]:
# Input: single file, directory, or default to data/
INPUT_PATH = PROJECT_ROOT / "data"  # Change to specific file if needed: PROJECT_ROOT / "data" / "validate.csv"

# Output paths
OUTPUT_DIR = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

COMBINED_OUTPUT = OUTPUT_DIR / "churn_scores_combined.csv"
SQL_VIEW_OUTPUT = OUTPUT_DIR / "churn_scores_sql_view.csv"

print(f"Input path: {INPUT_PATH}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Combined output: {COMBINED_OUTPUT}")
print(f"SQL view output: {SQL_VIEW_OUTPUT}")

## 3) Load Data

In [ ]:
def find_csv_files(input_path: Path) -> list[Path]:
    """Find CSV files from input path (file or directory)."""
    if not input_path.exists():
        raise FileNotFoundError(f"Path not found: {input_path}")
    
    if input_path.is_file():
        if input_path.suffix.lower() != '.csv':
            raise ValueError(f"Input file must be a CSV file: {input_path}")
        return [input_path]
    
    # It's a directory - find all CSV files
    csv_files: list[Path] = sorted(input_path.glob("*.csv"))
    
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in {input_path}")
    
    return csv_files

csv_files: list[Path] = find_csv_files(INPUT_PATH)
print(f"Found {len(csv_files)} CSV file(s) to process:")
for f in csv_files:
    print(f"  - {f.name}")

In [ ]:
# Load all CSV files
all_dataframes = []

for csv_file in csv_files:
    print(f"Loading {csv_file.name}...")
    df = pd.read_csv(csv_file, low_memory=False)
    df['SourceFile'] = csv_file.name  # Track source file
    all_dataframes.append(df)
    print(f"  ✓ Loaded {len(df):,} rows")

print(f"\nTotal files loaded: {len(all_dataframes)}")
print(f"Total rows: {sum(len(df) for df in all_dataframes):,}")

In [ ]:
# NOTE: Report generation happens after section 6 (after shaped_df is created)
# The generate_model_report function is defined in section 8
# It will be called after shaped_df is created in the "Apply shaping" cell


## 4) Score Customers

## 8) Generate Model Report

Create a comprehensive business-ready report analyzing the model performance and scored data.

In [ ]:
def generate_model_report(
    combined_df: pd.DataFrame,
    shaped_df: pd.DataFrame,
    model_obj,
    output_path: Path
) -> str:
    """
    Generate a comprehensive business-ready model report in markdown format.
    
    Args:
        combined_df: All scored records (all snapshots)
        shaped_df: Shaped data (latest snapshot per customer, SQL view format)
        model: Loaded XGBoost model
        output_path: Path to save the markdown report
    
    Returns:
        The generated report text as a string
    """
    import pandas as pd  # Ensure pd is available in function scope
    from datetime import datetime
    
    report_lines = []
    
    # Header
    report_lines.append("# Customer Churn Prediction Model - Business Report")
    report_lines.append("")
    report_lines.append(f"**Report Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report_lines.append("")
    report_lines.append("---")
    report_lines.append("")
    
    # Executive Summary
    report_lines.append("## Executive Summary")
    report_lines.append("")
    report_lines.append("This report provides a comprehensive analysis of the customer churn prediction model, ")
    report_lines.append("including model performance metrics, risk distribution analysis, and business insights ")
    report_lines.append("derived from the scored customer data.")
    report_lines.append("")
    
    total_customers = shaped_df['CustomerId'].nunique() if 'CustomerId' in shaped_df.columns else len(shaped_df)
    total_records = len(combined_df)
    
    report_lines.append(f"- **Total Customers Analyzed:** {total_customers:,}")
    report_lines.append(f"- **Total Records Scored:** {total_records:,}")
    
    if 'SnapshotDate' in combined_df.columns:
        date_range = f"{combined_df['SnapshotDate'].min()} to {combined_df['SnapshotDate'].max()}"
        report_lines.append(f"- **Date Range:** {date_range}")
    
    report_lines.append("")
    report_lines.append("---")
    report_lines.append("")
    
    # Model Information
    report_lines.append("## Model Information")
    report_lines.append("")
    report_lines.append("### Model Architecture")
    report_lines.append("")
    report_lines.append("- **Algorithm:** XGBoost (Gradient Boosting)")
    report_lines.append("- **Type:** Binary Classification (Churn Risk Prediction)")
    report_lines.append("- **Prediction Window:** 90-day churn risk")
    report_lines.append("")
    
    if model_obj is not None:
        try:
            # Get model parameters
            params = model_obj.get_params()
            report_lines.append("### Model Parameters")
            report_lines.append("")
            key_params = ['n_estimators', 'max_depth', 'learning_rate', 'subsample', 'colsample_bytree']
            for param in key_params:
                if param in params:
                    report_lines.append(f"- **{param.replace('_', ' ').title()}:** {params[param]}")
            report_lines.append("")
        except Exception:
            pass
    
    # Get feature count
    feature_count = len([c for c in combined_df.columns 
                        if c not in ['CustomerId', 'AccountName', 'Segment', 'CostCenter', 
                                    'SnapshotDate', 'ChurnRiskPct', 'RiskBand', 
                                    'Reason_1', 'Reason_2', 'Reason_3', 'SourceFile', 
                                    'ScoredAt', 'FirstPurchaseDate', 'LastPurchaseDate', 'Status']])
    report_lines.append(f"- **Number of Features:** {feature_count}")
    report_lines.append("")
    report_lines.append("---")
    report_lines.append("")
    
    # Data Overview
    report_lines.append("## Data Overview")
    report_lines.append("")
    report_lines.append("### Dataset Statistics")
    report_lines.append("")
    report_lines.append(f"- **Total Records:** {total_records:,}")
    report_lines.append(f"- **Unique Customers:** {total_customers:,}")
    
    if 'SourceFile' in combined_df.columns:
        num_files = combined_df['SourceFile'].nunique()
        report_lines.append(f"- **Source Files Processed:** {num_files}")
        if num_files > 1:
            report_lines.append("")
            report_lines.append("**Files Processed:**")
            file_counts = combined_df['SourceFile'].value_counts()
            for filename, count in file_counts.items():
                pct = (count / total_records) * 100
                report_lines.append(f"- {filename}: {count:,} records ({pct:.1f}%)")
    
    report_lines.append("")
    
    # Date range analysis
    if 'SnapshotDate' in combined_df.columns:
        report_lines.append("### Temporal Coverage")
        report_lines.append("")
        earliest = combined_df['SnapshotDate'].min()
        latest = combined_df['SnapshotDate'].max()
        report_lines.append(f"- **Earliest Snapshot:** {earliest}")
        report_lines.append(f"- **Latest Snapshot:** {latest}")
        
        if pd.api.types.is_datetime64_any_dtype(combined_df['SnapshotDate']):
            date_range_days = (pd.to_datetime(latest) - pd.to_datetime(earliest)).days
            report_lines.append(f"- **Time Span:** {date_range_days} days")
        report_lines.append("")
    
    report_lines.append("---")
    report_lines.append("")
    
    # Risk Distribution Analysis
    report_lines.append("## Risk Distribution Analysis")
    report_lines.append("")
    
    if 'RiskBand' in shaped_df.columns:
        risk_dist = shaped_df['RiskBand'].value_counts().sort_index()
        total = len(shaped_df)
        
        report_lines.append("### Risk Band Distribution")
        report_lines.append("")
        report_lines.append("| Risk Band | Count | Percentage |")
        report_lines.append("|-----------|-------|------------|")
        for band, count in risk_dist.items():
            pct = (count / total) * 100
            report_lines.append(f"| {band} | {count:,} | {pct:.2f}% |")
        report_lines.append("")
        
        # Interpretation
        report_lines.append("**Risk Band Definitions:**")
        report_lines.append("")
        report_lines.append("- **A (Low Risk):** Churn probability < 33%")
        report_lines.append("- **B (Medium Risk):** Churn probability 33-66%")
        report_lines.append("- **C (High Risk):** Churn probability > 66%")
        report_lines.append("")
    
    if 'ChurnRiskPct' in shaped_df.columns:
        report_lines.append("### Churn Risk Statistics")
        report_lines.append("")
        risk_stats = shaped_df['ChurnRiskPct'].describe()
        report_lines.append("| Metric | Value |")
        report_lines.append("|--------|-------|")
        report_lines.append(f"| Mean | {risk_stats['mean']:.4f} |")
        report_lines.append(f"| Median | {risk_stats['50%']:.4f} |")
        report_lines.append(f"| Standard Deviation | {risk_stats['std']:.4f} |")
        report_lines.append(f"| Minimum | {risk_stats['min']:.4f} |")
        report_lines.append(f"| Maximum | {risk_stats['max']:.4f} |")
        report_lines.append(f"| 25th Percentile | {risk_stats['25%']:.4f} |")
        report_lines.append(f"| 75th Percentile | {risk_stats['75%']:.4f} |")
        report_lines.append("")
        
        # High-risk customers
        high_risk_count = len(shaped_df[shaped_df['ChurnRiskPct'] >= 0.66])
        medium_risk_count = len(shaped_df[(shaped_df['ChurnRiskPct'] >= 0.33) & (shaped_df['ChurnRiskPct'] < 0.66)])
        low_risk_count = len(shaped_df[shaped_df['ChurnRiskPct'] < 0.33])
        
        report_lines.append("### Risk Segmentation")
        report_lines.append("")
        report_lines.append(f"- **High Risk (≥66%):** {high_risk_count:,} customers ({(high_risk_count/total_customers)*100:.1f}%)")
        report_lines.append(f"- **Medium Risk (33-66%):** {medium_risk_count:,} customers ({(medium_risk_count/total_customers)*100:.1f}%)")
        report_lines.append(f"- **Low Risk (<33%):** {low_risk_count:,} customers ({(low_risk_count/total_customers)*100:.1f}%)")
        report_lines.append("")
    
    report_lines.append("---")
    report_lines.append("")
    
    # Customer Status Analysis
    if 'Status' in shaped_df.columns:
        report_lines.append("## Customer Status Analysis")
        report_lines.append("")
        status_dist = shaped_df['Status'].value_counts()
        report_lines.append("### Status Distribution")
        report_lines.append("")
        report_lines.append("| Status | Count | Percentage |")
        report_lines.append("|--------|-------|------------|")
        for status, count in status_dist.items():
            pct = (count / total_customers) * 100
            report_lines.append(f"| {status} | {count:,} | {pct:.2f}% |")
        report_lines.append("")
        
        report_lines.append("**Status Definitions:**")
        report_lines.append("")
        report_lines.append("- **New:** Customer's first purchase within last 90 days")
        report_lines.append("- **Active:** Last purchase within last 90 days")
        report_lines.append("- **Churned:** Last purchase 91-180 days ago")
        report_lines.append("- **Reactivated:** Last purchase more than 180 days ago, but has recent activity")
        report_lines.append("")
        
        # Cross-tabulation: Status vs Risk
        if 'RiskBand' in shaped_df.columns:
            report_lines.append("### Status vs Risk Band Cross-Analysis")
            report_lines.append("")
            crosstab = pd.crosstab(shaped_df['Status'], shaped_df['RiskBand'], margins=True)
            report_lines.append("| Status | A (Low) | B (Medium) | C (High) | Total |")
            report_lines.append("|--------|---------|------------|----------|-------|")
            if crosstab.index is not None and len(crosstab.index) > 0:
                assert crosstab.index is not None  # Type guard
                index_values = crosstab.index  # Store in variable for type checker
                for status in index_values[:-1]:  # Exclude 'All' row
                    row = crosstab.loc[status]
                    all_value = row.get('All', 0)
                    report_lines.append(f"| {status} | {int(row.get('A', 0)):,} | {int(row.get('B', 0)):,} | {int(row.get('C', 0)):,} | {int(all_value):,} |")
            report_lines.append("")
        
        report_lines.append("---")
        report_lines.append("")
    
    # Key Performance Indicators
    report_lines.append("## Key Performance Indicators (KPIs)")
    report_lines.append("")
    
    kpis = []
    
    if 'ChurnRiskPct' in shaped_df.columns:
        avg_risk = shaped_df['ChurnRiskPct'].mean()
        kpis.append(("Average Churn Risk", f"{avg_risk:.2%}"))
        
        high_risk_pct = (len(shaped_df[shaped_df['ChurnRiskPct'] >= 0.66]) / total_customers) * 100
        kpis.append(("High Risk Customer Rate", f"{high_risk_pct:.2f}%"))
    
    if 'Status' in shaped_df.columns:
        churned_pct = (len(shaped_df[shaped_df['Status'] == 'Churned']) / total_customers) * 100
        kpis.append(("Churned Customer Rate", f"{churned_pct:.2f}%"))
        
        active_pct = (len(shaped_df[shaped_df['Status'] == 'Active']) / total_customers) * 100
        kpis.append(("Active Customer Rate", f"{active_pct:.2f}%"))
    
    if 'RiskBand' in shaped_df.columns and 'Status' in shaped_df.columns:
        # High-risk active customers (immediate action needed)
        high_risk_active = len(shaped_df[(shaped_df['RiskBand'] == 'C') & (shaped_df['Status'] == 'Active')])
        kpis.append(("High-Risk Active Customers (Action Required)", f"{high_risk_active:,}"))
    
    report_lines.append("| KPI | Value |")
    report_lines.append("|-----|-------|")
    for kpi_name, kpi_value in kpis:
        report_lines.append(f"| {kpi_name} | {kpi_value} |")
    report_lines.append("")
    report_lines.append("---")
    report_lines.append("")
    
    # Sample Data
    report_lines.append("## Sample Data")
    report_lines.append("")
    report_lines.append("### High-Risk Customers (Top 10)")
    report_lines.append("")
    
    if 'ChurnRiskPct' in shaped_df.columns:
        high_risk_sample = shaped_df.nlargest(10, 'ChurnRiskPct')[
            ['CustomerId', 'AccountName', 'ChurnRiskPct', 'RiskBand', 'Status'] + 
            (['Reason_1', 'Reason_2', 'Reason_3'] if all(c in shaped_df.columns for c in ['Reason_1', 'Reason_2', 'Reason_3']) else [])
        ]
        
        # Convert to markdown table
        report_lines.append("| Customer ID | Account Name | Risk % | Risk Band | Status | Top Reasons |")
        report_lines.append("|-------------|--------------|--------|-----------|--------|------------|")
        for _, row in high_risk_sample.iterrows():
            reasons = ""
            if 'Reason_1' in row:
                reasons = f"{row.get('Reason_1', 'N/A')[:50]}..."
            report_lines.append(f"| {row.get('CustomerId', 'N/A')} | {str(row.get('AccountName', 'N/A'))[:30]} | {row.get('ChurnRiskPct', 0):.2%} | {row.get('RiskBand', 'N/A')} | {row.get('Status', 'N/A')} | {reasons} |")
        report_lines.append("")
    
    report_lines.append("---")
    report_lines.append("")
    
    # Business Insights
    report_lines.append("## Business Insights & Recommendations")
    report_lines.append("")
    
    insights = []
    
    if 'RiskBand' in shaped_df.columns:
        high_risk_count = len(shaped_df[shaped_df['RiskBand'] == 'C'])
        if high_risk_count > 0:
            insights.append(f"**{high_risk_count:,} customers ({high_risk_count/total_customers*100:.1f}%) are classified as high-risk.** "
                          f"These customers require immediate attention and proactive retention efforts.")
    
    if 'Status' in shaped_df.columns and 'RiskBand' in shaped_df.columns:
        high_risk_active = len(shaped_df[(shaped_df['RiskBand'] == 'C') & (shaped_df['Status'] == 'Active')])
        if high_risk_active > 0:
            insights.append(f"**{high_risk_active:,} active customers are at high risk of churning.** "
                          f"This represents an opportunity for proactive intervention before they become inactive.")
    
    if 'ChurnRiskPct' in shaped_df.columns:
        avg_risk = shaped_df['ChurnRiskPct'].mean()
        if avg_risk > 0.5:
            insights.append(f"**The average churn risk across all customers is {avg_risk:.1%}.** "
                          f"This indicates a significant portion of the customer base may be at risk.")
    
    if 'Reason_1' in shaped_df.columns:
        # Most common reasons
        high_risk_df = shaped_df[shaped_df['RiskBand'] == 'C']
        if len(high_risk_df) > 0 and 'Reason_1' in high_risk_df.columns:
            reason_series: pd.Series = pd.Series(high_risk_df['Reason_1'])  # Ensure it's a Series
            top_reasons = reason_series.value_counts().head(3)
            if len(top_reasons) > 0:
                insights.append("**Top risk factors for high-risk customers:**")
            for i, (reason, count) in enumerate(top_reasons.items(), 1):
                insights.append(f"  {i}. {reason} ({count} customers)")
    
    for insight in insights:
        report_lines.append(insight)
        report_lines.append("")
    
    report_lines.append("### Recommended Actions")
    report_lines.append("")
    report_lines.append("1. **Immediate Action:** Contact high-risk active customers (Risk Band C, Status: Active) with retention offers")
    report_lines.append("2. **Monitoring:** Track medium-risk customers (Risk Band B) for early warning signs")
    report_lines.append("3. **Analysis:** Review top risk factors to identify common patterns and root causes")
    report_lines.append("4. **Engagement:** Develop targeted retention campaigns based on customer status and risk level")
    report_lines.append("")
    report_lines.append("---")
    report_lines.append("")
    
    # Technical Details
    report_lines.append("## Technical Details")
    report_lines.append("")
    report_lines.append("### Model Outputs")
    report_lines.append("")
    report_lines.append("- **ChurnRiskPct:** Probability of churn within 90 days (0.0 to 1.0)")
    report_lines.append("- **RiskBand:** Categorical risk classification (A=Low, B=Medium, C=High)")
    report_lines.append("- **Reason_1, Reason_2, Reason_3:** Top contributing factors to the churn risk prediction")
    report_lines.append("")
    report_lines.append("### Data Processing")
    report_lines.append("")
    report_lines.append("- **Scoring Method:** Batch processing of historical snapshots")
    report_lines.append("- **Output Format:** CSV files compatible with Power BI and SQL")
    report_lines.append("- **SQL View:** Data shaped to match `dbo.vwCustomerCurrent` view structure")
    report_lines.append("")
    
    # Write report
    report_text = "\n".join(report_lines)
    output_path.write_text(report_text, encoding='utf-8')
    
    print(f"✓ Model report generated: {output_path}")
    print(f"  Report length: {len(report_lines)} lines")
    
    return report_text

# Generate the report (run this after shaped_df is created in section 6)
# MODEL_REPORT_OUTPUT = OUTPUT_DIR / "model_report.md"
# print("Generating comprehensive model report...")
# print("This may take a moment...")
# report_text = generate_model_report(combined_df, shaped_df, model, MODEL_REPORT_OUTPUT)
# print(f"\n✓ Model report saved to: {MODEL_REPORT_OUTPUT}")
# print(f"  Open this file to view the business-ready analysis report")

In [ ]:
# Load model (this will be cached for subsequent scoring)
print("Loading model...")
model = load_model()
print("✓ Model loaded successfully")

In [ ]:
# Score each file and combine results
all_scored = []
start_time = time.time()

for idx, df in enumerate(all_dataframes, 1):
    file_start = time.time()
    source_file = df['SourceFile'].iloc[0]
    print(f"\n[{idx}/{len(all_dataframes)}] Scoring {source_file}...")
    
    # Score this dataframe
    scored_df = score_customers(df)
    
    # Add source file metadata (if not already present)
    if 'SourceFile' not in scored_df.columns:
        scored_df['SourceFile'] = source_file
    
    # Add processing timestamp
    scored_df['ScoredAt'] = datetime.now()
    
    all_scored.append(scored_df)
    
    file_time = time.time() - file_start
    print(f"  ✓ Scored {len(scored_df):,} records in {file_time:.2f} seconds")

# Combine all results
if len(all_scored) == 1:
    combined_df = all_scored[0]
else:
    print(f"\nCombining {len(all_scored)} files...")
    combined_df = pd.concat(all_scored, ignore_index=True)

total_time = time.time() - start_time
print(f"\n{'='*60}")
print(f"✓ Scoring completed in {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print(f"Total records scored: {len(combined_df):,}")
print(f"{'='*60}")

## 5) Explore Scored Results

In [ ]:
# Summary statistics
print("SCORING SUMMARY")
print("="*60)
print(f"Total records: {len(combined_df):,}")

if 'RiskBand' in combined_df.columns:
    print("\nRisk Band Distribution:")
    risk_dist = combined_df['RiskBand'].value_counts()
    for band, count in risk_dist.items():
        pct = (count / len(combined_df)) * 100
        print(f"  {band}: {count:,} ({pct:.1f}%)")

if 'ChurnRiskPct' in combined_df.columns:
    print("\nChurn Risk Statistics:")
    print(f"  Mean: {combined_df['ChurnRiskPct'].mean():.4f}")
    print(f"  Median: {combined_df['ChurnRiskPct'].median():.4f}")
    print(f"  Min: {combined_df['ChurnRiskPct'].min():.4f}")
    print(f"  Max: {combined_df['ChurnRiskPct'].max():.4f}")

if 'SnapshotDate' in combined_df.columns:
    print("\nDate Range:")
    print(f"  Earliest: {combined_df['SnapshotDate'].min()}")
    print(f"  Latest: {combined_df['SnapshotDate'].max()}")

if 'SourceFile' in combined_df.columns and combined_df['SourceFile'].nunique() > 1:
    print("\nFiles Processed:")
    file_counts = combined_df['SourceFile'].value_counts()
    for filename, count in file_counts.items():
        print(f"  {filename}: {count:,} records")

In [ ]:
# Preview the scored data
print("Preview of scored data:")
print(f"Columns: {len(combined_df.columns)}")
print(f"\nColumn names:")
for col in combined_df.columns:
    print(f"  - {col}")

print("\nFirst few rows:")
display(combined_df.head())

## 6) Shape for SQL View

In [ ]:
def calculate_status(row):
    """Calculate customer status like SQL fnCalculateStatus."""
    first_purchase = pd.to_datetime(row.get('FirstPurchaseDate'), errors='coerce')
    last_purchase = pd.to_datetime(row.get('LastPurchaseDate'), errors='coerce')
    snapshot = pd.to_datetime(row.get('SnapshotDate'), errors='coerce')
    
    if pd.isna(first_purchase) or pd.isna(last_purchase) or pd.isna(snapshot):
        return 'Unknown'
    
    # Days since first purchase
    days_since_first = (snapshot - first_purchase).days
    
    # Days since last purchase
    days_since_last = (snapshot - last_purchase).days
    
    if days_since_first <= 90:
        return 'New'
    elif days_since_last <= 90:
        return 'Active'
    elif days_since_last <= 180:
        return 'Churned'
    else:
        return 'Reactivated'

def shape_like_sql_view(df: pd.DataFrame) -> pd.DataFrame:
    """Shape data to match SQL view dbo.vwCustomerCurrent."""
    print("Shaping data to match SQL view structure...")
    
    # 1. Get latest snapshot per customer (like ROW_NUMBER window function)
    if 'SnapshotDate' in df.columns:
        df = df.sort_values(['CustomerId', 'SnapshotDate'], ascending=[True, False])
        df = df.drop_duplicates(subset=['CustomerId'], keep='first')
        print(f"  ✓ Latest snapshot per customer: {len(df):,} records")
    
    # 2. Calculate Status
    if 'FirstPurchaseDate' in df.columns and 'LastPurchaseDate' in df.columns:
        df['Status'] = df.apply(calculate_status, axis=1)
        print(f"  ✓ Calculated Status column")
    else:
        print("  ⚠ Warning: Missing FirstPurchaseDate or LastPurchaseDate - Status will be Unknown")
        df['Status'] = 'Unknown'
    
    # 3. Remove SourceFile (local processing artifact)
    if 'SourceFile' in df.columns:
        df = df.drop(columns=['SourceFile'])
        print(f"  ✓ Removed SourceFile column")
    
    # 4. Order columns to match SQL view (key columns first, then features, then scores)
    id_cols = ['CustomerId', 'AccountName', 'Segment', 'CostCenter', 'SnapshotDate']
    date_cols = ['FirstPurchaseDate', 'LastPurchaseDate']
    status_cols = ['Status']
    score_cols = ['ChurnRiskPct', 'RiskBand', 'Reason_1', 'Reason_2', 'Reason_3']
    
    # Build ordered column list
    ordered_cols = []
    for col_list in [id_cols, date_cols, status_cols, score_cols]:
        for col in col_list:
            if col in df.columns:
                ordered_cols.append(col)
    
    # Add remaining columns (features)
    remaining_cols = [c for c in df.columns if c not in ordered_cols and c != 'ScoredAt']
    ordered_cols.extend(sorted(remaining_cols))
    
    # Add ScoredAt at the end if present
    if 'ScoredAt' in df.columns:
        ordered_cols.append('ScoredAt')
    
    # Reorder columns
    df = df[ordered_cols]
    print(f"  ✓ Reordered columns to match SQL view structure")
    
    return df

print("✓ Shape functions defined")

In [ ]:
# Apply shaping
shaped_df = shape_like_sql_view(combined_df.copy())

print(f"\n{'='*60}")
print(f"Shaped data: {len(shaped_df):,} records")
print(f"{'='*60}")

# Show status distribution
if 'Status' in shaped_df.columns:
    print("\nStatus Distribution:")
    status_dist = shaped_df['Status'].value_counts()
    for status, count in status_dist.items():
        pct = (count / len(shaped_df)) * 100
        print(f"  {status}: {count:,} ({pct:.1f}%)")

print("\nPreview of shaped data:")
display(shaped_df.head())

# Generate the comprehensive model report
print("\n" + "="*60)
print("Generating comprehensive model report...")
MODEL_REPORT_OUTPUT = OUTPUT_DIR / "model_report.md"
report_text = generate_model_report(combined_df, shaped_df, model, MODEL_REPORT_OUTPUT)
print(f"\n✓ Model report saved to: {MODEL_REPORT_OUTPUT}")
print(f"  Open this file to view the business-ready analysis report")
print("="*60)

## 7) Export Outputs

In [ ]:
# Export combined scored data
print(f"Writing combined scored data to {COMBINED_OUTPUT}...")
combined_df.to_csv(COMBINED_OUTPUT, index=False)
file_size_mb = COMBINED_OUTPUT.stat().st_size / (1024 * 1024)
print(f"✓ Wrote {len(combined_df):,} records ({file_size_mb:.2f} MB)")

In [ ]:
# Export shaped data (SQL view format)
print(f"Writing shaped data to {SQL_VIEW_OUTPUT}...")
shaped_df.to_csv(SQL_VIEW_OUTPUT, index=False)
file_size_mb = SQL_VIEW_OUTPUT.stat().st_size / (1024 * 1024)
print(f"✓ Wrote {len(shaped_df):,} records ({file_size_mb:.2f} MB)")
print(f"\nThis file matches the structure of SQL view: dbo.vwCustomerCurrent")
print(f"Ready for Power BI exploration!")

In [ ]:
# Copy documentation files
readme_src = PROJECT_ROOT / "README.md"
if readme_src.exists():
    shutil.copy2(readme_src, OUTPUT_DIR / "README.md")
    print("✓ Copied README.md")

conda_src = PROJECT_ROOT / "model" / "conda.yml"
if conda_src.exists():
    shutil.copy2(conda_src, OUTPUT_DIR / "model_conda.yml")
    print("✓ Copied model/conda.yml")

print(f"\n{'='*60}")
print("✓ All outputs exported successfully!")
print(f"Output directory: {OUTPUT_DIR}")
print(f"{'='*60}")